- 대안 생성자
- 메모리 절약 slots / gc.collect()
<!-- - repr() / str()의 차이 -->

# Chapter9. 파이썬스러운 객체
> 절대로 결코! 앞에 언더바 두 개를 사용하지 말라. 이것은 짜증스러울 정도로 개인적인 이름이다. - 이안 비킹(Ian Bicking)

파이썬 데이터 모델 덕분에 사용자가 정의한 자료형도 내장 자료형과 마찬가지로 자연스럽게 동작할 수 있다. 상속하지 않고도 *덕 타이핑 매커니즘을 통해 모든 것이 가능하다.
사용자 정의 클래스를 만들어 보자.

사용자 정의 클래스를 만들기 위해서 다음과 같은방법을 설명한다.
- `repr()`, `bytes()` 등 객체를 다르 방식으로 표현하는 내장 함수의 지원
- 클래스 메서드로 대안 생성자 구현
- `format()` 내장 함수와 `str.format()` 메서드에서 사용하는 포맷 언어 확장
- 읽기 전용 접근만 허용하는 속성 제공
- 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
- `__slots__`를 이용해서 메모리 절약하기

<details> 
<summary> *덕 타이핑 (Duck Typing) </summary>
- 오리라고 부르면 오리가 됨)
```python
# what is duck typing? (https://en.wikipedia.org/wiki/Duck_typing)
# - 사람이 오리인척 하면 오리라고 봐도 된다? 
#   즉, 미리 타입을 정해놓지 않고, 실행 시점에 타입을 결정하는 것을 의미한다.
class Duck:
    def quack(self): 
        print("꽥꽥!")
    def feathers(self):
        print("오리에게 흰색, 회색 깃털이 있습니다.")

class Person:
    def quack(self): 
        print("이 사람이 오리를 흉내내네요.")
    def feathers(self):
        print("사람은 바닥에서 깃털을 주워서 보여 줍니다.")

def in_the_forest(duck):
    duck.quack()
    duck.feathers()


in_the_forest(Duck())
in_the_forest(Person())

--------------------------------------------------------------------------
꽥꽥!
오리에게 흰색, 회색 깃털이 있습니다.
이 사람이 오리를 흉내내네요.
사람은 바닥에서 깃털을 주워서 보여 줍니다.
```
</details>

## 유클리드 벡터형
- `@classmethod`와 `@staticmethod`를 언제 사용할 것인가?
- 파이썬에서 비공개 및 보호된 송성, 사용법, 관계, 한계

## 9.1 객체 표현

- `repr()` 내장 함수는 객체를 **개발자**가 보기 좋은 형태의 문자열로 변환한다.
- `str()` 내장 함수는 객체를 **사용자**가 보기 좋은 형태의 문자열로 변환한다.

In [114]:
from array import array
import math

class Vector:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):       # __iter__ 를 통해서 iterable 하게 만들어 준다. -> 언패킹 가능
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return f"repr-> {class_name}({self.x!r}, {self.y!r})"
        # return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return "str-> " + str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)       # hypot = sqrt(x**2 + y**2)) 빗변의 길이를 구함 (즉 벡터간의 거리를 의미함)

    def __bool__(self):
        return bool(abs(self))

In [119]:
v1 = Vector(3, 4)

print(v1.x, v1.y)

3.0 4.0


In [120]:
print(*v1)      # *v1 을 통해서 언패킹 가능

3.0 4.0


In [121]:
x, y = v1
x, y

(3.0, 4.0)

In [122]:
print(v1)       # print는 str을 호출한다.

str-> (3.0, 4.0)


In [134]:
v1

repr-> Vector(3.0, 4.0)

In [129]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [80]:
abs(v1)

5.0

In [133]:
bool(v1), bool(Vector(0,0))

(True, False)

In [73]:
for x in v1:
    print(x)

3.0
4.0


In [132]:
print(v1 == [3,4])      # 내부에서 tuple로 처리하기 때문에 True 반환
print(v1 == (3,4))

True
True


### Appendix: @dataclass
<!-- https://hwiyong.tistory.com/420 -->

python 3.7부터는 @dataclass 라는 데코레이터를 통해 간단하게 dataclass를 만들 수 있다.  
(doc: https://docs.python.org/ko/3/library/dataclasses.html)
```python
from dataclasses import dataclass

@dataclass
class something:
    object: str
    value: int

a = something("obj1", 1)
b = something("obj2", 2)
```

In [182]:
from dataclasses import dataclass

@dataclass
class something:
    object: str
    value: int

a = something("obj1", 1)
b = something("obj2", 2)

print(a)
print(b)

something(object='obj1', value=1)
something(object='obj2', value=2)


## 9.3 대안 생성자
Vector를 bytes로 변환하는 메서드가 있으니, byte를 Vecotr로 변환할 수 있으면 좋겠다.  
array.array의 표준 라이브러리에서 frombytes()라는 메서드를 사용해 byte를 Vector로 만드는 메서드를 클래스 메서드로 추가해보자.
```python
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)
```

In [174]:
from array import array
import math

class Vector:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):       # __iter__ 를 통해서 iterable 하게 만들어 준다. -> 언패킹 가능
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return f"repr-> {class_name}({self.x!r}, {self.y!r})"
        # return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return "str-> " + str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)       # hypot = sqrt(x**2 + y**2)) 빗변의 길이를 구함 (즉 벡터간의 거리를 의미함)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [179]:
v1_clone = Vector.frombytes(bytes(v1))
v1_clone

3.0 4.0
3.0
4.0


repr-> Vector(3.0, 4.0)

In [178]:
v1 == v1_clone

True